In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as ex
import bz2
import func as F
import torch

In [2]:
ae_res_path = "G:/MEX/results/ae_results.pbz2"
ref_res_path = "G:/MEX/results/Reference_results.pbz2"
trans_res_path = "G:/MEX/results/transfer_results.pbz2"
ref_reduc_res_path = "G:/MEX/results/transfer_results_reduc_ref.pbz2"
ref_reduc_res_path2 = "G:/MEX/results/transfer_results_reduc_ref_2-5.pbz2"
reduc_res_path = "G:/MEX/results/transfer_results_reduc.pbz2"


In [5]:
ae_res = F.load(ae_res_path)


In [46]:
dfs = []
for r1 in ae_res:
    r1_res = pd.DataFrame()
    r1_res["name"] = pd.Series([r1["name"]])
    r1_res["params"] = pd.Series([r1["params"]])
    r1_res["mem"] = pd.Series([r1["mem"]])
    r1_res["time"] = pd.Series([np.mean(r1["elapsed_times"])])

    recon_err = 0
    for err in r1["recon_error"]:
        recon_err += err
    recon_err /= float(len(r1["recon_error"]))
    r1_res["recon_err"] = pd.Series([recon_err.item()])

    adv_err = 0
    for err in r1["adv_error"]:
        adv_err += err
    recon_err /= float(len(r1["adv_error"]))
    r1_res["adv_err"] = pd.Series([adv_err.item()])

    rot_err = 0
    for err in r1["rot_error"]:
        rot_err += err
    rot_err /= float(len(r1["rot_error"]))
    r1_res["rot_err"] = pd.Series([rot_err.item()])

    sx = [sum(abs(delta[0])).item() for delta in r1["delta_rot"]]
    sy = [sum(abs(delta[1])).item() for delta in r1["delta_rot"]]
    sxx = sum(sx) / float(len(sx))
    syy = sum(sy) / float(len(sy))

    r1_res["sum_delta_rot_x"] = pd.Series([sxx])
    r1_res["sum_delta_rot_y"] = pd.Series([syy])
    dfs.append(r1_res)


In [47]:
F.save(dfs, "ae_dfs", "../results")

In [3]:
ref_res = F.load(ref_res_path)

In [20]:
dfs = extract(ref_res)

F.save(dfs, "ref_dfs", "../results")

In [9]:
dfs[0].keys()

dict_keys(['AE+MoE', 'AE+LSTM', 'RBF-CAT+LSTM', 'RBF-IN+LSTM', 'RBF-CAT+MoE', 'RBF-IN+MoE', 'VAE-CAT+LSTM', 'VAE-IN+LSTM', 'VAE-CAT+MoE', 'VAE-IN+MoE', 'DEC-CAT+MoE', 'DEC-IN+MoE'])

In [10]:
print(len(dfs[0]["AE+MoE"]["recon_err"]))


1


In [19]:
def extract(data):
    df = []
    for r1 in data:
        ref_df = {}
        for k, v in r1.items():
            r1_res = pd.DataFrame()
            # r1_res["name"] = pd.Series([v["name"]])
            # r1_res["params"] = pd.Series([v["params"]])
            # r1_res["mem"] = pd.Series([v["mem"]])
            # r1_res["time"] = pd.Series([np.mean(v["elapsed_times"])])

            recon_err = [err.item() for err in v["recon_error"]]
            r1_res["recon_err"] = pd.Series(np.asarray(recon_err))
            adv_err = [err.item() for err in v["adv_error"]]
            r1_res["adv_err"] = pd.Series(np.asarray(adv_err))

            rot_err = [err.item() for err in v["rot_error"]]
            r1_res["rot_err"] = pd.Series(np.asarray(rot_err))
            sx = [sum(abs(delta[0])).item() for delta in v["delta_rot"]]
            sy = [sum(abs(delta[1])).item() for delta in v["delta_rot"]]
            sxx = pd.Series(np.asarray(sx))
            syy = pd.Series(np.asarray(sy))

            r1_res["sum_delta_rot_x"] = pd.Series([sxx])
            r1_res["sum_delta_rot_y"] = pd.Series([syy])
            pCostx = [(sum(cost[0])/float(len(cost[0])+1)) for cost in v["potCost"]]
            pCosty = [(sum(cost[1])/float(len(cost[1])+1)) for cost in v["potCost"]]
            rCostx = [(sum(cost[0])/float(len(cost[3])+1)) for cost in v["rotCost"]]
            rCosty = [(sum(cost[1])/float(len(cost[2])+1)) for cost in v["rotCost"]]

            r1_res["avg_sum_pCost_x"] = pd.Series(np.asarray(pCostx))
            r1_res["avg_sum_pCost_y"] = pd.Series(np.asarray(pCosty))
            r1_res["avg_sum_rCost_x"] = pd.Series(np.asarray(rCostx))
            r1_res["avg_sum_rCost_t"] = pd.Series(np.asarray(rCosty))

            ref_df[k] = r1_res
        df.append(ref_df)
    return df

In [22]:
transfer_res = F.load(trans_res_path)
dfs = extract(transfer_res)
F.save(dfs, "transfer_dfs", "../results")

KeyboardInterrupt: 

In [ ]:
reduct_res1 = F.load(ref_reduc_res_path)
reduct_res2 = F.load(ref_reduc_res_path2)

In [ ]:
reduc_res = [reduct_res1[0]] + reduct_res2

In [ ]:
dfs = extract(reduc_res)
F.save(dfs, "ref_reduc_dfs", "../results")

In [ ]:
reduc_res_tr = F.load(reduc_res_path)
dfs = extract(reduc_res_tr)
F.save(dfs, "reduc_dfs", "../results")